In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from gaze_utility import calculate_dist_and_veloc
from fixation import generate_IVT_fixation
from preprocessing import remove_outlier_by_z, get_participant_names

data_path = 'data/'
exp_img_path = 'exp-img/'
xn = 'GazeX'
yn = 'GazeY'

# get participant names
names = get_participant_names(data_path)
print('participants names: ', names)
print('participant num: ', len(names))

participants names:  ['yz', 'yq', 'hw', 'yx', 'lf', 'huy', 'hu', 'xiao', 'jt', 'mr', 'dao', 'lr', 'mh', 'an', 'ts', 'co']
participant num:  16


In [2]:
# generate intermediate file
from generate_experiment_img import *
from rank_utility import *
# generate grid img
# click_rank_files = [f for f in os.listdir(data_path) if "click_rank" in f]
# for file_name in click_rank_files:
#     generate_grid_iamge(get_grid_image_order(data_path, file_name))
# generate rank
# - click rank
click_rank_files = [f for f in os.listdir(data_path) if "click_rank" in f]
generate_click_rank(data_path, click_rank_files)
# - general rank
pairwise_rank_files = [f for f in os.listdir(data_path) if "pairwise_rank" in f]
generate_pairwise_rank(data_path, pairwise_rank_files)

FileNotFoundError: [Errno 2] No such file or directory: './rank_result/click_rank_result.csv'

In [2]:
# take grid1 of for example
name = 'mr'
exp = 'grid_1'
exp_img = mpimg.imread(exp_img_path + name + '_' + exp + '.jpg')
gaze_data = pd.read_json(data_path + name + '_gaze.txt', lines=True)
data = gaze_data.loc[gaze_data['record_event'] == 'gaze_grid_2']
sns.scatterplot(x=xn, y=yn, data=data, alpha=0.2)

FileNotFoundError: [Errno 2] No such file or directory: 'exp-img/mr_grid_1.jpg'

In [ ]:
data.head()

In [ ]:
# remove outlier
data = remove_outlier_by_z(data, xn)
data = remove_outlier_by_z(data, yn)
sns.scatterplot(x=xn, y=yn, data=data, alpha=0.2)

In [ ]:
# calculate dist, velocity, time passed
data['distance'], data['velocity'], data['time_pass'] = calculate_dist_and_veloc(data)

In [ ]:
v = data['velocity'].round(1)
print(v.value_counts())
sns.distplot(v)

In [ ]:
fixations = generate_IVT_fixation(data, v_th=0.5, d_th=300)
fixations_points = np.array([f['centroid'] for f in fixations])
durations = [f['duration'] for f in fixations]

# fixation point
plot = sns.scatterplot(x=fixations_points[:, 0], y=fixations_points[:, 1],
                size=300, sizes=(100,200),
                hue=durations, alpha=0.8,
                legend=False)
plt.axis('off')

# background image
plt.imshow(exp_img, zorder=0, alpha=0.5,
           aspect=plot.get_aspect(),
           extent=plot.get_xlim() + plot.get_ylim())

# draw grid
(x_min, x_max) = plot.get_xlim()
(y_min, y_max) = plot.get_ylim()
# - horizontal
hlines_y = [y_min, (y_min + y_max)/2, y_max]
plt.hlines(hlines_y, xmin=x_min, xmax=x_max,
           zorder=1, alpha=0.3)
# - vertical
n = (x_max - x_min) / 3
plt.vlines([round(x_min + i*n, 4) for i in range(4)],
           ymin=y_min, ymax=y_max,
           zorder=1, alpha=0.3)

# scan path
for i, f in enumerate(fixations_points):
    if i + 1 == len(fixations): break
    nex = fixations_points[i+1]
    cur = fixations_points[i]
    plt.arrow(cur[0], cur[1], nex[0]-cur[0], nex[1]-cur[1],
              alpha=0.2, head_length=50, head_width=25,
              zorder=100, length_includes_head=True)
    # annotation of order
    x_anno = (cur[0] + nex[0])/2
    y_anno = (cur[1] + nex[1])/2
    b_props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(x=x_anno, y=y_anno, s=i+1, backgroundcolor='white', bbox=b_props, size=6)


In [ ]:
sns.distplot(durations)

# Batch Analysis

In [7]:
data_path = 'data/'
result_path = 'result/'
events = ['gaze_grid_1', 'gaze_ranking_grid_1', 'gaze_grid_2', 'gaze_ranking_grid_2']
exps = ['gaze_grid_1', 'gaze_grid_2']
xn = 'GazeX'
yn = 'GazeY'

# get participant names
names = get_participant_names(data_path)
print('participants names: ', names)
print('participant num: ', len(names))

for name in names:
    gaze_data = pd.read_json(data_path + name + '_gaze.txt', lines=True)
    for event in events:
        # log
        print('start: ' + name + '-' + event)
        # load data
        data = gaze_data.loc[gaze_data['record_event'] == event]
        # remove outlier
        data = remove_outlier_by_z(data, xn)
        data = remove_outlier_by_z(data, yn)
        # construct attr
        data['distance'], data['velocity'], data['time_pass'] = calculate_dist_and_veloc(data)
        # generate fixation
        fixations = generate_IVT_fixation(data, v_th=0.5, d_th=300)
        fixations_points = np.array([f['centroid'] for f in fixations])
        durations = [f['duration'] for f in fixations]
        # draw fixation
        sns.scatterplot(x=fixations_points[:, 0], y=fixations_points[:, 1],
                size=durations, sizes=(100,200),
                hue=durations, alpha=0.8)
        # draw scan path
        for i, f in enumerate(fixations_points):
            if i + 1 == len(fixations): break
            nex = fixations_points[i+1]
            cur = fixations_points[i]
            plt.arrow(cur[0], cur[1], nex[0]-cur[0], nex[1]-cur[1],
                    alpha=0.2, head_length=50, head_width=25,
                    zorder=100, length_includes_head=True)
        # export result
        plt.savefig(result_path + name + '_' + event + '_plot.png', format='png')
        # log
        print('finish: ' + name + '-' + event)
        plt.clf()

participants names:  ['lf', 'yx', 'mr', 'jt', 'yz', 'ts', 'lr', 'yq', 'hw', 'mh']
participant num:  10
start: lf-gaze_grid_1
finish: lf-gaze_grid_1
start: lf-gaze_ranking_grid_1
finish: lf-gaze_ranking_grid_1
start: lf-gaze_grid_2
finish: lf-gaze_grid_2
start: lf-gaze_ranking_grid_2
finish: lf-gaze_ranking_grid_2
start: yx-gaze_grid_1
finish: yx-gaze_grid_1
start: yx-gaze_ranking_grid_1
finish: yx-gaze_ranking_grid_1
start: yx-gaze_grid_2
finish: yx-gaze_grid_2
start: yx-gaze_ranking_grid_2
finish: yx-gaze_ranking_grid_2
start: mr-gaze_grid_1
finish: mr-gaze_grid_1
start: mr-gaze_ranking_grid_1
finish: mr-gaze_ranking_grid_1
start: mr-gaze_grid_2
finish: mr-gaze_grid_2
start: mr-gaze_ranking_grid_2
finish: mr-gaze_ranking_grid_2
start: jt-gaze_grid_1
finish: jt-gaze_grid_1
start: jt-gaze_ranking_grid_1
finish: jt-gaze_ranking_grid_1
start: jt-gaze_grid_2
finish: jt-gaze_grid_2
start: jt-gaze_ranking_grid_2
finish: jt-gaze_ranking_grid_2
start: yz-gaze_grid_1
finish: yz-gaze_grid_1
star

<Figure size 432x288 with 0 Axes>